### Importing libraries

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import csv

### Ingesting Data

In [108]:
train = pd.read_csv('datasets/train.csv', index_col='Id', keep_default_na=False,
                                                       # na_values=
                                                        )
test =  pd.read_csv('datasets/test.csv', index_col='Id', keep_default_na=False,
                                                        #na_values=
                                                        )
#data = pd.concat([train, test])
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000


In [109]:
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,Inside,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
1464,60,RL,78,9978,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
1465,120,RL,43,5005,Pave,NA,IR1,HLS,AllPub,Inside,...,144,0,NA,NA,NA,0,1,2010,WD,Normal


In [110]:
data_description_reader = pd.read_csv('datasets/data_description.txt',
                                sep=r'[\t:]',
                                na_filter=False,
                                header=None,
                                names = ['variable','description'],
                                # index_col = 0,
                                on_bad_lines='skip',
                                skip_blank_lines=False,
                                skipinitialspace=False,
                                # iterator=True,
                                # chunksize=1
                                

)
# data_description_reader.__module__


C:\Users\gylbt\AppData\Local\Temp\ipykernel_336\617450481.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_description_reader = pd.read_csv('datasets/data_description.txt',


In [111]:
data_description_reader 

,variable,description
0,MSSubClass,Identifies the type of dwelling involved in t...
1,20,1-STORY 1946 & NEWER ALL STYLES
2,30,1-STORY 1945 & OLDER
3,40,1-STORY W/FINISHED ATTIC ALL AGES
4,45,1-1/2 STORY - UNFINISHED ALL AGES
...,...,...
465,Abnorml,"Abnormal Sale - trade, foreclosure, short sale"
466,AdjLand,Adjoining Land Purchase
467,Alloca,Allocation - two linked properties with...
468,Family,Sale between family members


In [112]:
breaks = data_description_reader.query('description.isna()').index.to_list()
vars = []
for brk in breaks:
    vars.append(data_description_reader.at[brk+1, 'variable'] )

vars.insert(0, data_description_reader.at[0, 'variable'])
data_description_reader.insert(1, 'category', 'see')

In [114]:
i=0
x=0
for brk in breaks:
    print(f'row {i}:{brk-1}')
    data_description_reader.loc[i: brk, 'category'] = vars[x]
    i=breaks[x]+1
    x+=1

data_description_reader


row 0:16


KeyboardInterrupt: 

### Data Exploration

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.select_dtypes('object').astype('string')
obj_cols  = train.select_dtypes('object').astype('string').columns.to_list()
repl = []
for col in obj_cols:
    if train[col].str.isnumeric().sum() > 100:
        print(col) 
        repl.append(col)

### Change to float

In [ ]:
for col in repl:
    train[col].replace('NA', np.nan, inplace=True)
    test[col].replace('NA', np.nan, inplace=True)
    train[col] = train[col].astype(float)
    test[col] = test[col].astype(float)
    


In [ ]:
train.info()

### Variable Distribution

In [ ]:
train.columns

In [ ]:
# grid = sns.FacetGrid(train)
# grid.map_dataframe(sns.histplot)
# plt.savefig('histograms.jpg')

## Missing Data

In [ ]:
def check_missing(df=train):
    missing = df.isna().sum()
    missing_data = missing[missing > 0]
    return missing_data

check_missing()

In [ ]:
msno.matrix(train)

In [ ]:
msno.heatmap(train)

# Fill Missing Data

### YearBuilt

In [ ]:
corr = train[["GarageYrBlt", "YearBuilt"]].corr()
corr

In [ ]:
train["GarageYrBlt"].fillna(train["YearBuilt"],inplace=True)
test["GarageYrBlt"].fillna(test["YearBuilt"],inplace=True)

## LotFrontage

In [ ]:
check_missing()

In [ ]:
# sns.catplot(data=train, x='LotFrontage',kind='strip', row='LotConfig', sharex=False,height=3,orient='portrait')

# plt.show()


In [ ]:
lot_front_dic = train.mask(train.LotFrontage.isna()) \
            .groupby('LotConfig') \
                ['LotFrontage'].median().round() \
                .to_dict()

lot_front_dic

In [ ]:
# missing_LF = train[(train.LotConfig == key) & (train.LotFrontage.isna())]  

for key, item in lot_front_dic.items():
    train.loc[((train.LotConfig==key ) & (train.LotFrontage.isna())), 'LotFrontage'] = item
    # test.loc[((train.LotConfig==key ) & (train.LotFrontage.isna())), 'LotFrontage'] = item

In [ ]:
check_missing(train)

In [ ]:
train.info()

In [ ]:
train.to_csv('datasets/train_clean.csv')
test.to_csv('datasets/test_clean.csv')

# Drop Columns

### Drop 8 rows of ManVnrArea

In [ ]:
train.dropna(axis = 'rows', subset=["MasVnrArea"], inplace=True)

### Drop categorical columns  with more than 600 missing values

In [ ]:
# t = int(train.shape[0]*.90)
# train.dropna(axis='columns', thresh= t, inplace=True)

### Drop Columns Dominated by a certain value

In [ ]:
# unbalanced_cols = []
# for col in train.select_dtypes('object').columns:
#     value_counts = train[col].value_counts(normalize=True)

#     max = value_counts.max()
    
#     print(value_counts)
#    # print(max)

#     if max > .70:
#         unbalanced_cols.append(col)


In [ ]:
# train.drop(unbalanced_cols, axis='columns',inplace=True)
# check_missing()

In [ ]:
# train.dropna(axis=0,inplace=True)
# print(train.shape)
# check_missing()

# Feature Importance

## Correlation of Predictors with Target Variables

In [ ]:
# def plot_correlation_heatmap(target_col: str or None):

#     corr_data: pd.DataFrame
    
#     if target_col is None:
corr_price = train.corr(method='pearson', numeric_only=True)['SalePrice'].sort_values(ascending=False)
#     elif isinstance(target_col, str):
#         corr_data = train.corr(method='pearson')

sns.heatmap(corr_price.to_frame())
plt.show()

# plot_correlation_heatmap('SalePrice')

## Correlation between Predictor Variables (Multicollinearity)

In [ ]:
corr_df = train.corr(method='pearson', numeric_only=True).sort_values(by='SalePrice')
fig = plt.figure(figsize=(10,12))
sns.heatmap(corr_df,cmap='BrBG', figure=fig )
plt.show()

In [ ]:
corr_df


## Drop highly correlated independent variables

In [ ]:
to_drop = corr_df.query('OverallCond >-.80 and OverallCond >.80').index.tolist()
to_drop.append('OverallQual')
train.drop(columns=to_drop, inplace=True)
train

# Modeling

## Imports

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score


In [ ]:
c =test.select_dtypes(include='object').columns.to_list()
d = train.select_dtypes(include='object').columns.to_list()

## Preprocessing Pipeline

## Encoding

In [ ]:
data_description


In [ ]:
train.query('Alley == Street')[['Alley','Street']].shape[0]

## Baseline Model (Linear Regression)    

In [ ]:
linear_reg = LinearRegression()

linear_reg.fit(x_train, y_train)

In [ ]:
for col in test.columns:
    if col not in x_train.columns:
        test.drop(axis='column', columns=col, inplace=True)
        


In [ ]:
X_test_final = scale_encode(test)
check_missing(X_test_final)

In [ ]:
X_test_final.dropna(inplace=True)
y_pred = linear_reg.predict(X_test_final)
pd.Series(y_pred).to_csv('submission.csv')

## Baseline Model (Decision Tree)

In [ ]:
dt = DecisionTreeRegressor(criterion='squared_error')

dt.fit(x_train, y_train)

In [ ]:
dt.feature_names_in_

In [ ]:
dt.predict(X_test_final)